#### 1) Write a python program to display all the header tags from wikipedia.org and make data frame.

In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def scrape_wikipedia_headers(url):
    
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content)
    
    headers = []
    
    for i in soup.find_all('span',class_="mw-headline"):
        headers.append(i.text)
    
    df = pd.DataFrame(headers, columns=['Headers'])
    
    return df


In [4]:
url = 'https://en.wikipedia.org/wiki/Main_Page'
df = scrape_wikipedia_headers(url)
print(df)


                         Headers
0           Welcome to Wikipedia
1  From today's featured article
2               Did you know ...
3                    In the news
4                    On this day
5       Today's featured picture
6       Other areas of Wikipedia
7    Wikipedia's sister projects
8            Wikipedia languages


#### 2) Write s python program to display list of respected former presidents of India(i.e. Name , Term of office)
from https://presidentofindia.nic.in/former-presidents.htm and make data frame.

In [279]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def president_names(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    names = []
    terms = []
    
    for i in soup.find_all('div', class_="presidentListing"):
        text = i.text.strip()
        parts = text.split('\nTerm of Office: ')
        if len(parts) == 2:
            name, term = parts
            term = term.split(' (')[0]  # Remove the link part
        else:
            name, term = parts[0], None
        names.append(name)
        terms.append(term)

    df = pd.DataFrame({'Presidents': names, 'Term of Office': terms})
    return df

url = 'https://presidentofindia.nic.in/former-presidents.htm'
df = president_names(url)
print(df)



                                     Presidents  \
0           Shri Ram Nath Kovind (birth - 1945)   
1             Shri Pranab Mukherjee (1935-2020)   
2   Smt Pratibha Devisingh Patil (birth - 1934)   
3            DR. A.P.J. Abdul Kalam (1931-2015)   
4            Shri K. R. Narayanan (1920 - 2005)   
5           Dr Shankar Dayal Sharma (1918-1999)   
6               Shri R Venkataraman (1910-2009)   
7                  Giani Zail Singh (1916-1994)   
8         Shri Neelam Sanjiva Reddy (1913-1996)   
9          Dr. Fakhruddin Ali Ahmed (1905-1977)   
10     Shri Varahagiri Venkata Giri (1894-1980)   
11                 Dr. Zakir Husain (1897-1969)   
12     Dr. Sarvepalli Radhakrishnan (1888-1975)   
13             Dr. Rajendra Prasad (1884-1963)    

                                       Term of Office  
0   25 July, 2017 to 25 July, 2022 \nhttps://ramna...  
1   25 July, 2012 to 25 July, 2017 \nhttp://pranab...  
2   25 July, 2007 to 25 July, 2012 \nhttp://pratib...  
3   25 Jul

#### 3) Write a python program to scrape cricket rankings from icc-cricket.com. You have to scrape and make data frame
#### a) Top 10 ODI teams in men’s cricket along with the records for matches, points and rating

In [216]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def top_mteam(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    ranking=[]
    # Get the first ranking from a different class
    first_ranking_element = soup.find('td', class_='rankings-block__banner--pos')
    first_ranking = first_ranking_element.text.strip()
    ranking.append(first_ranking)
    
    # Get the remaining rankings
    ranking_elements = soup.find_all('td', class_='table-body__cell table-body__cell--position u-text-right')
    for r in ranking_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ranking.append(r.text.strip())
     
    teams = []
    for i, team in enumerate(soup.find_all('span', class_='u-hide-phablet'), 1):
        if i > 10:
            break
        teams.append(team.text.strip())
    
    matches=[]
    # Get the first matches from a different class
    first_matches_element = soup.find('td', class_='rankings-block__banner--matches')
    first_matches = first_matches_element.text.strip()
    matches.append(first_matches)
        
    points=[]
    # Get the first points from a different class
    first_points_element = soup.find('td', class_='rankings-block__banner--points')
    first_points = first_points_element.text.strip()
    points.append(first_points)
    
    table_rows = soup.find_all('tr')

    for row in table_rows[1:]:
        columns = row.find_all('td', class_='table-body__cell u-center-text')

        if len(columns) >= 2:
            matches.append(columns[0].text.strip())
            points.append(columns[1].text.strip())
            if len(matches) == 10 and len(points) == 10:
                break
                
    ratings = []
    
    # Get the first rating from a different class
    first_rating_element = soup.find('td', class_='rankings-block__banner--rating u-text-right')
    first_rating = first_rating_element.text.strip()
    ratings.append(first_rating)
    
    # Get the remaining ratings
    rank_elements = soup.find_all('td', class_="table-body__cell u-text-right rating")
    for rank in rank_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ratings.append(rank.text.strip())
    
    df = pd.DataFrame({'Rank':ranking,'Teams':teams,'Matches': matches,'Points':points,'Rating':ratings})
    return df

url="https://www.icc-cricket.com/rankings/mens/team-rankings/odi"
df = top_mteam(url)
print(df)


  Rank         Teams Matches Points Rating
0    1     Australia      23  2,714    118
1    2      Pakistan      20  2,316    116
2    3         India      33  3,807    115
3    4   New Zealand      27  2,806    104
4    5       England      24  2,426    101
5    6  South Africa      19  1,910    101
6    7    Bangladesh      25  2,451     98
7    8   Afghanistan      10    878     88
8    9     Sri Lanka      21  1,682     80
9   10   West Indies      25  1,797     72


#### 3) b) Top 10 ODI Batsmen along with the records of their team and rating.

In [237]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def top_batsmen(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

        
    ranking=[]
    # Get the first ranking from a different class
    first_ranking_element = soup.find('span', class_='rankings-block__pos-number')
    first_ranking = first_ranking_element.text.strip()
    ranking.append(first_ranking)
    
    # Get the remaining rankings
    ranking_elements = soup.find_all('span', class_="rankings-table__pos-number")
    for r in ranking_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ranking.append(r.text.strip())
    
    player=[]
     # Get the first player from a different class
    first_player_element = soup.find('div', class_="rankings-block__banner--name-large")
    first_player = first_player_element.text.strip()
    player.append(first_player)
    
    # Get the remaining players
    player_elements = soup.find_all('td', class_="table-body__cell rankings-table__name name")
    for p in player_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        player.append(p.text.strip())
    
    teams=[]
    # Get the first player team from a different class
    first_team_element = soup.find('div', class_="rankings-block__banner--nationality")
    first_team = first_team_element.text.strip()
    teams.append(first_team)
    
    # Get the remaining teams
    team_elements = soup.find_all('span', class_='table-body__logo-text')
    for t in team_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        teams.append(t.text.strip())
    
   
    ratings = []
    
    # Get the first rating from a different class
    first_rating_element = soup.find('div', class_='rankings-block__banner--rating')
    first_rating = first_rating_element.text.strip()
    ratings.append(first_rating)
    
    # Get the remaining ratings
    rank_elements = soup.find_all('td', class_="table-body__cell rating")
    for rank in rank_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ratings.append(rank.text.strip())
    
    df = pd.DataFrame({'Rank': ranking,'Player':player,"Team":teams,'Rating':ratings})
    return df

url = "https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting"
df = top_batsmen(url)
print(df)


  Rank                 Player Team Rating
0    1             Babar Azam  PAK    886
1    2  Rassie van der Dussen   SA    777
2    3           Fakhar Zaman  PAK    755
3    4            Imam-ul-Haq  PAK    745
4    5           Shubman Gill  IND    738
5    6           David Warner  AUS    726
6    7           Harry Tector  IRE    722
7    8            Virat Kohli  IND    719
8    9        Quinton de Kock   SA    718
9   10           Rohit Sharma  IND    707


#### 3) c) Top 10 ODI bowlers along with the records of their team andrating.

In [239]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def top_bowler(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

        
    ranking=[]
    # Get the first ranking from a different class
    first_ranking_element = soup.find('span', class_='rankings-block__pos-number')
    first_ranking = first_ranking_element.text.strip()
    ranking.append(first_ranking)
    
    # Get the remaining rankings
    ranking_elements = soup.find_all('span', class_="rankings-table__pos-number")
    for r in ranking_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ranking.append(r.text.strip())
    
    player=[]
     # Get the first player from a different class
    first_player_element = soup.find('div', class_="rankings-block__banner--name-large")
    first_player = first_player_element.text.strip()
    player.append(first_player)
    
    # Get the remaining players
    player_elements = soup.find_all('td', class_="table-body__cell rankings-table__name name")
    for p in player_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        player.append(p.text.strip())
    
    teams=[]
    # Get the first player team from a different class
    first_team_element = soup.find('div', class_="rankings-block__banner--nationality")
    first_team = first_team_element.text.strip()
    teams.append(first_team)
    
    # Get the remaining teams
    team_elements = soup.find_all('span', class_='table-body__logo-text')
    for t in team_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        teams.append(t.text.strip())
    
   
    ratings = []
    
    # Get the first rating from a different class
    first_rating_element = soup.find('div', class_='rankings-block__banner--rating')
    first_rating = first_rating_element.text.strip()
    ratings.append(first_rating)
    
    # Get the remaining ratings
    rank_elements = soup.find_all('td', class_="table-body__cell rating")
    for rank in rank_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ratings.append(rank.text.strip())
    
    df = pd.DataFrame({'Rank': ranking,'Player':player,"Team":teams,'Rating':ratings})
    return df

url = "https://www.icc-cricket.com/rankings/mens/player-rankings/odi/bowling"
df = top_bowler(url)
print(df)


  Rank            Player Team Rating
0    1    Josh Hazlewood  AUS    705
1    2    Mohammed Siraj  IND    691
2    3    Mitchell Starc  AUS    686
3    4        Matt Henry   NZ    667
4    5       Trent Boult   NZ    660
5    6       Rashid Khan  AFG    659
6    7        Adam Zampa  AUS    652
7    8  Mujeeb Ur Rahman  AFG    637
8    9     Mohammad Nabi  AFG    631
9   10    Shaheen Afridi  PAK    630


#### 4) Write a python program to scrape cricket rankings from icc-cricket.com. You have to scrape and make data frame

#### a) Top 10 ODI teams in women’s cricket along with the records for matches, points and rating.


In [207]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def top_wteam(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    ranking=[]
    # Get the first ranking from a different class
    first_ranking_element = soup.find('td', class_='rankings-block__banner--pos')
    first_ranking = first_ranking_element.text.strip()
    ranking.append(first_ranking)
    
    # Get the remaining rankings
    ranking_elements = soup.find_all('td', class_="table-body__cell table-body__cell--position u-text-right")
    for r in ranking_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ranking.append(r.text.strip())
    
    teams = []
    for i, team in enumerate(soup.find_all('span', class_="u-hide-phablet"), 1):
        if i > 10:
            break
        teams.append(team.text.strip())
    ratings = []
    
    # Get the first rating from a different class
    first_rating_element = soup.find('td', class_='rankings-block__banner--rating u-text-right')
    first_rating = first_rating_element.text.strip()
    ratings.append(first_rating)
    
    # Get the remaining ratings
    rank_elements = soup.find_all('td', class_="table-body__cell u-text-right rating")
    for rank in rank_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ratings.append(rank.text.strip())
    
    matches=[]
    # Get the first matches from a different class
    first_matches_element = soup.find('td', class_='rankings-block__banner--matches')
    first_matches = first_matches_element.text.strip()
    matches.append(first_matches)
        
    points=[]
    # Get the first points from a different class
    first_points_element = soup.find('td', class_='rankings-block__banner--points')
    first_points = first_points_element.text.strip()
    points.append(first_points)
    
    table_rows = soup.find_all('tr')

    for row in table_rows[1:]:
        columns = row.find_all('td', class_='table-body__cell u-center-text')

        if len(columns) >= 2:
            matches.append(columns[0].text.strip())
            points.append(columns[1].text.strip())
            if len(matches) == 10 and len(points) == 10:
                break
   
    df = pd.DataFrame({'Rank':ranking,'Teams': teams,'Matches': matches,'Points':points,'Rating':ratings})
    return df

url="https://www.icc-cricket.com/rankings/womens/team-rankings/odi"
df = top_wteam(url)
print(df)


  Rank         Teams Matches Points Rating
0    1     Australia      21  3,603    172
1    2       England      28  3,342    119
2    3  South Africa      26  3,098    119
3    4         India      27  2,820    104
4    5   New Zealand      25  2,553    102
5    6   West Indies      27  2,535     94
6    7      Thailand      11    821     75
7    8    Bangladesh      14    977     70
8    9      Pakistan      27  1,678     62
9   10     Sri Lanka       9    479     53


#### 4) b) Top 10 women’s ODI Batting players along with the records of their team and rating.

In [240]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def top_batsmen(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

        
    ranking=[]
    # Get the first ranking from a different class
    first_ranking_element = soup.find('span', class_='rankings-block__pos-number')
    first_ranking = first_ranking_element.text.strip()
    ranking.append(first_ranking)
    
    # Get the remaining rankings
    ranking_elements = soup.find_all('span', class_="rankings-table__pos-number")
    for r in ranking_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ranking.append(r.text.strip())
    
    player=[]
     # Get the first player from a different class
    first_player_element = soup.find('div', class_="rankings-block__banner--name-large")
    first_player = first_player_element.text.strip()
    player.append(first_player)
    
    # Get the remaining players
    player_elements = soup.find_all('td', class_="table-body__cell rankings-table__name name")
    for p in player_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        player.append(p.text.strip())
    
    teams=[]
    # Get the first player team from a different class
    first_team_element = soup.find('div', class_="rankings-block__banner--nationality")
    first_team = first_team_element.text.strip()
    teams.append(first_team)
    
    # Get the remaining teams
    team_elements = soup.find_all('span', class_='table-body__logo-text')
    for t in team_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        teams.append(t.text.strip())
    
   
    ratings = []
    
    # Get the first rating from a different class
    first_rating_element = soup.find('div', class_='rankings-block__banner--rating')
    first_rating = first_rating_element.text.strip()
    ratings.append(first_rating)
    
    # Get the remaining ratings
    rank_elements = soup.find_all('td', class_="table-body__cell rating")
    for rank in rank_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ratings.append(rank.text.strip())
    
    df = pd.DataFrame({'Rank': ranking,'Player':player,"Team":teams,'Rating':ratings})
    return df

url = "https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting"
df = top_batsmen(url)
print(df)


  Rank               Player Team Rating
0    1          Beth Mooney  AUS    754
1    2      Laura Wolvaardt   SA    732
2    3       Natalie Sciver  ENG    731
3    4          Meg Lanning  AUS    717
4    5     Harmanpreet Kaur  IND    716
5    6      Smriti Mandhana  IND    714
6    7  Chamari Athapaththu   SL    673
7    8         Ellyse Perry  AUS    626
8    9       Tammy Beaumont  ENG    595
9   10      Stafanie Taylor   WI    588


#### 4) c) Top 10 women’s ODI all-rounder along with the records of their team and rating.

In [241]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def allrounder(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

        
    ranking=[]
    # Get the first ranking from a different class
    first_ranking_element = soup.find('span', class_='rankings-block__pos-number')
    first_ranking = first_ranking_element.text.strip()
    ranking.append(first_ranking)
    
    # Get the remaining rankings
    ranking_elements = soup.find_all('span', class_="rankings-table__pos-number")
    for r in ranking_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ranking.append(r.text.strip())
    
    player=[]
     # Get the first player from a different class
    first_player_element = soup.find('div', class_="rankings-block__banner--name-large")
    first_player = first_player_element.text.strip()
    player.append(first_player)
    
    # Get the remaining players
    player_elements = soup.find_all('td', class_="table-body__cell rankings-table__name name")
    for p in player_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        player.append(p.text.strip())
    
    teams=[]
    # Get the first player team from a different class
    first_team_element = soup.find('div', class_="rankings-block__banner--nationality")
    first_team = first_team_element.text.strip()
    teams.append(first_team)
    
    # Get the remaining teams
    team_elements = soup.find_all('span', class_='table-body__logo-text')
    for t in team_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        teams.append(t.text.strip())
    
   
    ratings = []
    
    # Get the first rating from a different class
    first_rating_element = soup.find('div', class_='rankings-block__banner--rating')
    first_rating = first_rating_element.text.strip()
    ratings.append(first_rating)
    
    # Get the remaining ratings
    rank_elements = soup.find_all('td', class_="table-body__cell rating")
    for rank in rank_elements[0:9]:  # Start from the second rank and go up to the tenth rank
        ratings.append(rank.text.strip())
    
    df = pd.DataFrame({'Rank': ranking,'Player':player,"Team":teams,'Rating':ratings})
    return df

url = "https://www.icc-cricket.com/rankings/womens/player-rankings/odi/all-rounder"
df = allrounder(url)
print(df)

  Rank             Player Team Rating
0    1    Hayley Matthews   WI    373
1    2     Natalie Sciver  ENG    371
2    3       Ellyse Perry  AUS    366
3    4     Marizanne Kapp   SA    349
4    5        Amelia Kerr   NZ    336
5    6      Deepti Sharma  IND    322
6    7   Ashleigh Gardner  AUS    292
7    8      Jess Jonassen  AUS    250
8    9           Nida Dar  PAK    232
9   10  Sophie Ecclestone  ENG    205


#### 5) Write a python program to scrape mentioned news details from https://www.cnbc.com/world/?region=world and make data frame
i) Headline
ii) Time
iii) News Link

In [253]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def cnbc_news(url):
    
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content)
    
    headlines=[]
    
    for i in soup.find_all('a',class_="LatestNews-headline"):
        headlines.append(i.get('title'))
    
    time = []
    
    for i in soup.find_all('time',class_="LatestNews-timestamp"):
        time.append(i.text)
    link=[]
    for i in soup.find_all('a',class_="LatestNews-headline"):
        link.append(i.get('href'))

    df = pd.DataFrame({'Headline':headlines,'Time': time,'News Link':link})
    return df
    
    
url = 'https://www.cnbc.com/world/?region=world'
df = cnbc_news(url)
print(df)



                                             Headline          Time  \
0   State Farm to stop accepting homeowners insura...   3 Hours Ago   
1   How Janie Deegan built Janie's Life-Changing B...   8 Hours Ago   
2   Top 10 cheapest places in the U.S. to buy a be...   8 Hours Ago   
3   Mark Cuban calls Elon Musk’s Twitter algorithm...   8 Hours Ago   
4   3 investing tips as the federal debt ceiling '...   9 Hours Ago   
5   Microsoft keyboard users are ‘devastated’ afte...   9 Hours Ago   
6   Steve Adcock: The 3 'stupidest' myths I've hea...   9 Hours Ago   
7   Chip stocks AMD and Nvidia are among the most ...  10 Hours Ago   
8   Analysts are pounding the table for these must...  10 Hours Ago   
9   Lower-income consumers pay for wealthy's credi...  10 Hours Ago   
10               How Mastercard has outperformed Visa  10 Hours Ago   
11  Why commercial real estate firms are joining t...  10 Hours Ago   
12  Japan stocks are on fire this year. Why the ra...  10 Hours Ago   
13  Ma

#### 6) Write a python program to scrape the details of most downloaded articles from AI in last 90 days.https://www.journals.elsevier.com/artificial-intelligence/most-downloaded-articles

Scrape below mentioned details and make data frame
i) Paper Title
ii) Authors
iii) Published Date
iv) Paper URL

In [258]:

import requests
import pandas as pd
from bs4 import BeautifulSoup

def top_article(url):
    
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content)
    
    title=[]
    
    for i in soup.find_all('h2',class_="sc-1qrq3sd-1 gRGSUS sc-1nmom32-0 sc-1nmom32-1 btcbYu goSKRg"):
        title.append(i.text)
    
    authors=[]
    for i in soup.find_all('span',class_="sc-1w3fpd7-0 dnCnAO"):
        authors.append(i.text)
    date = []
    
    for i in soup.find_all('span',class_="sc-1thf9ly-2 dvggWt"):
        date.append(i.text)
    link=[]
    for i in soup.find_all('a',class_="sc-5smygv-0 fIXTHm"):
        link.append(i.get('href'))

    df = pd.DataFrame({'Title':title,"Authors":authors,"Published date":date,"URL":link})
    return df
    
    
url = 'https://www.journals.elsevier.com/artificial-intelligence/most-downloaded-articles'
df = top_article(url)
print(df)


                                                Title  \
0                                    Reward is enough   
1   Explanation in artificial intelligence: Insigh...   
2              Creativity and artificial intelligence   
3   Conflict-based search for optimal multi-agent ...   
4   Knowledge graphs as tools for explainable mach...   
5   Law and logic: A review from an argumentation ...   
6   Between MDPs and semi-MDPs: A framework for te...   
7   Explaining individual predictions when feature...   
8       Multiple object tracking: A literature review   
9   A survey of inverse reinforcement learning: Ch...   
10  Evaluating XAI: A comparison of rule-based and...   
11  Explainable AI tools for legal reasoning about...   
12            Hard choices in artificial intelligence   
13  Assessing the communication gap between AI mod...   
14  Explaining black-box classifiers using post-ho...   
15  The Hanabi challenge: A new frontier for AI re...   
16              Wrappers for fe

#### 7) Write a python program to scrape mentioned details from dineout.co.inand make data frame
i) Restaurant name
ii) Cuisine
iii) Location
iv) Ratings
v) Image URL

In [ ]:
# I am unable to work with dineout.co.in as I live in the UK. I am using deliveroo.uk instead

In [274]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def deliveroo(url):
    
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content)
    
    name=[]
    
    for i in soup.find_all('p',class_='ccl-649204f2a8e630fd ccl-a396bc55704a9c8a ccl-ff5caa8a6f2b96d0 ccl-40ad99f7b47f3781'):
        name.append(i.text)
    
    offers=[]
    
    for i in soup.find_all('div','HomeFeedUICard-d38caa5cc97794b4'):
        offers.append(i.text)
    
        
    df = pd.DataFrame({'Restaurant Name':name,"Special Offers":offers})
    return df
    
    
url = 'https://deliveroo.co.uk/restaurants/london/woodford/?geohash=u10j83y9myd1&collection=basket+discounts'
df = deliveroo(url)
print(df)

                               Restaurant Name              Special Offers
0                                 PIZZAEXPRESS    40%‬  off selected items
1                        Family Kebab Fish Bar    Spend £30Add a free item
2                                 SoBe Burger   Spend £15Get free delivery
3                                 Chick'N'Bun   Spend £15Get free delivery
4             Little Bao Boy @ The Spark House        Spend £20Get 20% off
5                            The Wing Brothers        Spend £20Get 10% off
6                           Pizza Hut Delivery           Buy 1, get 1 free
7                                    Chaiiwala        Spend £20Get 20% off
8                                    Patty Guy        Spend £20Get 20% off
9                            Cookies and Cream  Spend £20Get free delivery
10                                Grill Hustle  Spend £20Get free delivery
11                         Salamis Kebab House  Spend £15Get free delivery
12                      E